# Sentiment Analysis

Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

In [ ]:
from requests import get
import re

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import nltk

from nltk.sentiment import SentimentIntensityAnalyzer

## Sentiment from Text

The purpose of **text analysis** is to extract meaning from text data. This involves cleaning and processing text data, as well as using analysis methods that are able to get something **quantitative** out of something that doesn't inherently have **numbers**. So far, we've used **topic modeling** to do some of this to get an idea of what topics were discussed within documents. 

Another way to extract meaning from text is by assigning values of **sentiment**. The words we use have meaning, and we can assign measures of what they are intended to portray. For example, the word "bad" is generally a negative sentiment (slang usage notwithstanding), while "good" has a positive sentiment. The word "hurt" generally also has a negative sentiment, while "heal" has a positive one. In this way, we can attempt to put different words onto the same scale and measure the overall sentiment of text.

In this section, we will look at doing a type of analysis called **sentiment analysis**, which is a class of techniques designed to extract this type of meaning from text data. In particular, we'll look at one dictionary-based method called **VADER** (Valence Aware Dictionary and Sentiment Reasoner). 

## Example with Twitter Data

VADER is a **dictionary-based** method, meaning it is pre-built and comes with a list of words and scores. To use it, we need to download the list of words with scores, then apply those scores to the words within our documents. Combining the scores of the words/tokens within our document gives us the overall sentiment of the document. For VADER, we will get back the negative, neutral, positive, and compound scores of a document.

To use VADER, we first download the `vader_lexicon` resource. 

In [ ]:
nltk.download('vader_lexicon')

VADER was actually developed for and meant to be used with social media data, such as with Twitter. That is, the dictionary scores that are part of VADER are generally developed with shorter posts and some slang in mind. The `nltk` package actually comes with some sample twitter data to test these methods out on. 

In [ ]:
nltk.download('twitter_samples')

We'll first do some light cleaning of the data to make any existing links unclickable. The `tweets` object below should be a list with all tweets from the sample dataset provided with `nltk`.

In [ ]:
# make it so that we can't accidentally click on links
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]

In [ ]:
tweets[:10]

To do sentiment analysis with VADER, we first create a `SentimentIntensityAnalyzer` object. This works similarly to how we did topic modeling with Latent Dirichlet Allocation. We then provide the data that we want scored. 

In [ ]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(tweets[0])

The scores for negative, neutral, and positive are always positive, and indicate how much of that type of sentiment is present in the document. The compound score is a value from -1 to 1 that provides an overall summary of how positive or negative that document is in its sentiment. 

The compound score is most often used, and typically, the threshold for being considered positive, neutral, or negative is as follows:
- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05

<font color ='red'>**Question 1: Create a list called `sentiments` that contains the compound sentiment for each tweet in `tweets`.**</font>

We can also do some basic analyses of the sentiment scores. For example, we can find some summary statistics, as well as create graphs of the distribution of the sentiment score.

In [ ]:
plt.hist(sentiments, bins = 20)

<font color ='red'>**Question 2: What is the mean compound sentiment in `sentiments`?**</font>

## Data - NYT 2021 Archive

Though VADER was developed for shorter social media posts, we can still use it for other types of text as well. It generally works best with shorter documents, though, and longer forms of text such as movie reviews would be better served by breaking them apart and looking at sentiment of individual sentences.

The abstract data that we have been working with from the New York Times API is relatively short and can actually work quite well for this type of analysis. As before, let's bring in all articles in 2021 from the NYT Archive.

In [ ]:
nyt_2021 = pd.read_csv('nyt_2021.csv').dropna()

In [ ]:
nyt_2021.head()

Let's take a quick look at one abstract to see what it would be scored as.

In [ ]:
sia.polarity_scores(nyt_2021.abstract[0])

Let's take a look at the abstract to see what it actually says.

In [ ]:
nyt_2021.abstract[0]

<font color ='red'>**Question 3: Find the compound score for each abstract in `nyt_2021`. Create a new column in `nyt_2021` called `sentiment_score` that contains the compound score.**</font>

## Using Sentiment Scores

Now that we have calculated sentiment scores for our article abstracts, we can look at summaries and try to understand more about the abstracts using various summary statistics and graphs.

In [ ]:
nyt_2021.sentiment_score.describe()

In [ ]:
nyt_2021.sentiment_score.hist()

We can also use them to look at trends and differences across different types of articles and types of content. For example, let's take a look at all Op-Ed articles about Biden. 

In [ ]:
biden_op_df = nyt_2021[(nyt_2021.type_of_material == 'Op-Ed') & (nyt_2021.abstract.str.contains('Biden'))]

To get an idea of the sentiment of the Op-Eds written about Biden, we can first create a quick histogram. 

In [ ]:
biden_op_df.sentiment_score.hist()

Most of them seemed to be relatively neutral, but there are peaks around -0.5 and 0.5 indicating articles that were positive or negative. This makes sense for Op-Eds since they are opinion pieces, so we might expect stronger positive or negative language. 

If we wanted to see trends, we could also look at the sentiment over time. Let's take a look at the average sentiment for these articles by month. The seaborn `sns.lineplot` does the aggregation for us and even provides error bars to get an idea of what the range of sentiments might be.

In [ ]:
sns.lineplot(biden_op_df, x = 'month', y = 'sentiment_score')

The sentiment generally seems to be neutral or positive with a quick drop in August. Why might this be the case? This is around the time that Biden announced the withdrawal of US troops from Afghanistan, which drew lots of criticism for how it was handled. Let's take a look at some of the abstracts to see if that's what we see. 

In [ ]:
biden_op_df[biden_op_df.month == 8]

<font color ='red'>**Question 4: What was the overall trend of the sentiment of News articles over the course of 2021? Did the overall trend differ for articles that mentioned "Biden"?**</font>

## Other Types of Sentiment

VADER only uses positive, negative, or neutral for its sentiment scores. However, there are other types of sentiment possible. For example, there might be more nuanced version of positive or negative, such as happy or excited or anger or fear. There are other types of sentiment analysis that do this as well. The state of the art in terms of sentiment analysis (and other types of text analysis) is using neural networks. We'll go over some examples of this type of analysis in the coming weeks. 